In [2]:
from pyspark.ml.linalg import SparseVector
from operator import attrgetter

df = sc.parallelize([
    (SparseVector(3, [0, 2], [1.0, 3.0]), ),
    (SparseVector(3, [1], [4.0]), )
]).toDF(["features"])

features = df.rdd.map(attrgetter("features"))

In [3]:
indexed_features = features.zipWithIndex()

In [4]:
def explode(row):
    vec, i = row
    for j, v in zip(vec.indices, vec.values):
        yield i, j, v

entries = indexed_features.flatMap(explode)

In [5]:
row_indices, col_indices, data = zip(*entries.collect())

In [6]:
shape = (
    df.count(),
    df.rdd.map(attrgetter("features")).first().size
)

In [7]:
from scipy.sparse import csr_matrix

mat = csr_matrix((data, (row_indices, col_indices)), shape=shape)

In [8]:
mat.todense()

matrix([[ 1.,  0.,  3.],
        [ 0.,  4.,  0.]])

In [9]:
import numpy as np

def as_matrix(vec):
    data, indices = vec.values, vec.indices
    shape = 1, vec.size
    return csr_matrix((data, indices, np.array([0, vec.values.size])), shape)

mats = features.map(as_matrix)

In [13]:
from scipy.sparse import vstack

mat = mats.reduce(lambda x, y: vstack([x, y]))



In [12]:
type(mat)

scipy.sparse.csr.csr_matrix